<a href="https://colab.research.google.com/github/dataista0/twitch-streams/blob/main/Episodio_5_Agregamos_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Episodio 5 - Notebook de Colab



## Estas dos primeras celdas son de setup, solo hace falta correrlas una vez al iniciar el entorno

In [3]:
!pip install transformers
!git clone https://github.com/dataista0/twitch-streams.git

     |████████████████████████████████| 2.5MB 19.1MB/s 
     |████████████████████████████████| 901kB 50.1MB/s 
     |████████████████████████████████| 3.3MB 50.8MB/s 


In [ ]:
from google.colab import files
files.upload() # Para subir la api key (kaggle.json)

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!mkdir -p twitch-streams/data/nlp-getting-started
!kaggle competitions download -c nlp-getting-started -p data/nlp-getting-started/

In [1]:
import sys; sys.path.append("twitch-streams")
import utils
import importlib
importlib.reload(utils)

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from utils import get_submission, submit, get_tokenizer_and_model,\
                  compute_metrics, load_dfs, tokenize, get_train_args


import transformers
from transformers import Trainer
transformers.logging.set_verbosity_warning()

def read_scores():
    s = !kaggle competitions submissions nlp-getting-started 
    df = pd.DataFrame([(l.split()[0], l.split()[-2]) for l in s[2:]], columns=["Archivo", "Score"]).set_index("Archivo")
    display(df)
    return df

In [2]:
model_id = "episodio_5_colab_1"
pred_file_name = f"{model_id}.csv"
submit_message = "Episodio 5 Roberta Large desde Colab"

model_name = "roberta-large"
tokenizer, model = get_tokenizer_and_model(model_name)

df_base, df_test = load_dfs()
df_train, df_val = train_test_split(df_base, test_size=0.066)
ds_train, ds_val, ds_test = tokenize(tokenizer, df_train, df_val, df_test)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classi

In [3]:
trainer = Trainer(model=model, args=get_train_args(model_id, epochs=1, batch_size=4, save_steps=500), 
                  train_dataset=ds_train, 
                  eval_dataset=ds_val,
                  compute_metrics=compute_metrics)

trainer.train()

***** Running training *****
  Num examples = 7110
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1778


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.691800,0.669554,0.638171,0.000000,0.000000,0.000000


Saving model checkpoint to /content/twitch-streams/data/models/episodio_5_colab_1/checkpoint-500
Configuration saved in /content/twitch-streams/data/models/episodio_5_colab_1/checkpoint-500/config.json
Model weights saved in /content/twitch-streams/data/models/episodio_5_colab_1/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/twitch-streams/data/models/episodio_5_colab_1/checkpoint-1000
Configuration saved in /content/twitch-streams/data/models/episodio_5_colab_1/checkpoint-1000/config.json
Model weights saved in /content/twitch-streams/data/models/episodio_5_colab_1/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/twitch-streams/data/models/episodio_5_colab_1/checkpoint-1500
Configuration saved in /content/twitch-streams/data/models/episodio_5_colab_1/checkpoint-1500/config.json
Model weights saved in /content/twitch-streams/data/models/episodio_5_colab_1/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 503
  B

TrainOutput(global_step=1778, training_loss=0.700210442559121, metrics={'train_runtime': 2791.8456, 'train_samples_per_second': 2.547, 'train_steps_per_second': 0.637, 'total_flos': 7761783875604480.0, 'train_loss': 0.700210442559121, 'epoch': 1.0})

In [4]:
df_res = get_submission(trainer, ds_test, pred_file_name)

***** Running Prediction *****
  Num examples = 3263
  Batch size = 4


KeyboardInterrupt: ignored

In [ ]:
submit(pred_file_name, submit_message)

In [ ]:
# Leaderbord - ??
read_scores();